### Necessary functions

In [1]:
import requests
import pandas as pd
from datetime import datetime as dt

pd.set_option('display.max_colwidth', None)
URL="https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

In [2]:
page=requests.get(URL)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
poke_content=soup.find(id='mw-content-text')
poke_tables=poke_content.find_all('table')

In [5]:
def getAllPokemons():
    for i in range(1, 9):
        getPokemons(i)

In [6]:
def getGenPokemons(i):
    getPokemons(i)

In [7]:
def getPokemons(i):
    select_generation = i
    gen_list = poke_tables[select_generation]
    poke_info=gen_list.contents[1]
    
    len_elements = []
    for each in poke_info:
        if each != '\n':
            len_elements.append(len(each.text.strip().split('\n')))
            
    select_element_num = 11
    for each in poke_info:
        if each != '\n':
            if len(each.text.strip().split('\n')) == select_element_num:
                print(each.text.strip().split('\n'))
                
    info_start = 1
    # place where to get the pokemon info
    info_row=gen_list.contents[info_start]

    for pokemon_info_values, even_index_chec in zip(info_row.contents, range(0,len(info_row.contents))):
        # Pokemons' values are stored in even index (divisible by 2 and is not 0)
        if ((even_index_chec % 2) == 0) & (even_index_chec != 0) :
            pokemon_raw_info = pokemon_info_values.text.strip().split('\n')
            link_start = "https://bulbapedia.bulbagarden.net/wiki/"
            now = dt.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")


            ## Pokemons that are other regional forms and has 1 type
            if len(pokemon_raw_info) == 7:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = ''
                generation = i
                link = link_start + poke_name
                scrapetime = dt_string

            ## Pokemons that are other regional forms and has 2 types
            elif len(pokemon_raw_info) == 8:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = pokemon_raw_info[7]
                generation = i
                link = link_start + poke_name
                scrapetime = dt_string

            ## Orig pokemon list with single typing
            elif len(pokemon_raw_info) == 9:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = ''
                generation = i
                link = link_start + poke_name
                scrapetime = dt_string


            # Orig pokemon list with multiple typing
            elif len(pokemon_raw_info) == 10:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = pokemon_raw_info[9]
                generation = i
                link = link_start + poke_name
                scrapetime = dt_string

            else:
                print('Check out elements containing ' + str(len(pokemon_raw_info)) + ' elements')

            # Saving as a tuple
            extracted_poke_info.append((kdex, ndex, poke_name, type1, type2, link, generation, scrapetime))


        else:
            pass

### The fun starts here!

In [8]:
extracted_poke_info = []
print('Would you like to extract all pokemons (Yes/No)?')
answer = input()
if(answer == 'Yes' or answer == 'yes'):
    getAllPokemons()
else:
    print('Which generation do you want to extract (1-8)?')
    gen_input = input()
    if(int(gen_input) < 1 or int(gen_input) > 8):
        print('Invalid input. Try again.')
    else:
        getGenPokemons(int(gen_input))

Would you like to extract all pokemons (Yes/No)?
yes


### Transfer data frame to json file

In [9]:
df_pokemon_list = pd.DataFrame(extracted_poke_info)
df_pokemon_list.columns = ['Kdex', 'Ndex', 'Pokemon', 'Type 1', 'Type 2', 'Link', 'Generation', 'Scrapetime']
df_pokemon_list.to_json('ExtractedPokemons.json', orient='index')

### Read generated json file

In [10]:
dfPokemon = pd.read_json('ExtractedPokemons.json', orient='index')
dfPokemon

,Kdex,Ndex,Pokemon,Type 1,Type 2,Link,Generation,Scrapetime
0,#001,#001,Bulbasaur,Grass,Poison,https://bulbapedia.bulbagarden.net/wiki/Bulbasaur,1,18/05/2022 02:37:40
1,#002,#002,Ivysaur,Grass,Poison,https://bulbapedia.bulbagarden.net/wiki/Ivysaur,1,18/05/2022 02:37:40
2,#003,#003,Venusaur,Grass,Poison,https://bulbapedia.bulbagarden.net/wiki/Venusaur,1,18/05/2022 02:37:40
3,#004,#004,Charmander,Fire,,https://bulbapedia.bulbagarden.net/wiki/Charmander,1,18/05/2022 02:37:40
4,#005,#005,Charmeleon,Fire,,https://bulbapedia.bulbagarden.net/wiki/Charmeleon,1,18/05/2022 02:37:40
...,...,...,...,...,...,...,...,...
991,#---,#901,Ursaluna,Ground,Normal,https://bulbapedia.bulbagarden.net/wiki/Ursaluna,8,18/05/2022 02:37:40
992,#---,#902,Basculegion,Water,Ghost,https://bulbapedia.bulbagarden.net/wiki/Basculegion,8,18/05/2022 02:37:40
993,#---,#903,Sneasler,Fighting,Poison,https://bulbapedia.bulbagarden.net/wiki/Sneasler,8,18/05/2022 02:37:40
994,#---,#904,Overqwil,Dark,Poison,https://bulbapedia.bulbagarden.net/wiki/Overqwil,8,18/05/2022 02:37:40
